# wellpathpy prototype

## Goal

To build a light package to load well deviations

## Objectives

1. load well deviation in one of <_n_> formats:
    * meta data (header, rkb, dfe, rt)
    * md, incl, azi
    * mE, mN, depth
    * other ?,?,?
2. interpolate survey using one of these methods:
    * minimum curvature method
    * radius of curvature method
    * tangential method
    * other ?
3. calculate dog-leg severity
4. calculate depth references using header data if available: MD, TVD, TVDSS
5. return interpolated deviation in all <_n_> input formats and in all depth references if possible
6. resample on regular steps

## Sources of equations:

- [petrowiki](https://petrowiki.org/Calculation_methods_for_directional_survey)
- [Crain's Petrophysical Handbook](https://www.spec2000.net/19-dip13.htm)
- [drillingformulas](http://www.drillingformulas.com)

## software _architecture_

(get input from Jørgen: `jokva`)

- code to `list` (will then also accept `nd.array`)
- data reader (chooses files to accept: `*.csv`, `*.xlsx`)
- guess data types if not provided by user?
- library of interpolation functions
- depth calculation function
- deploy to [pypi](https://pypi.org/)

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('../data/Well_Surveys_Projected_to_TD.csv', skiprows=4, usecols=['MD[m]',
                                                                                 'Inc[deg]',
                                                                                 'Azi[deg]',
                                                                                 'Dogleg [deg/30m]',
                                                                                 'TVD[m]',
                                                                                 'North[m]',
                                                                                 'East[m]'],
                )
df.head()

In [ ]:
df.drop(df.tail(1).index,inplace=True)
df.dropna(inplace=True)
df.to_csv('../data/deviation.csv', index=False)

In [ ]:
def read_deviation(fname):
    """
    Read a deviation file into memory for processing
    
    Args:
        filename
        
    Return:
        pd.DataFrame
        
    """
    return pd.read_csv(fname)


In [ ]:
read_deviation('../data/deviation.csv').head()

In [ ]:
def get_header(datum='kb', units='m', elevation=0.):
    """
    Record deviation header information needed for depth
    reference calculation into DataFrame.
    
    Parameters
    ----------
    datum: str, default 'kb', {'kb', 'dfe', 'rt'}
           'kb' (kellybushing),
           'dfe' (drill floor elevation),
           'rt' (rotary table)
    units: str, default 'm', {'m', 'ft'}
           'm' (metres),
           'ft' (feet)
    elevation: float, default 0., 
           <datum> <elevation> in <units> 
           above mean sea level
    
    Returns
    -------
    dict
        deviation header dictionnary
    """
    if datum not in {'kb', 'dfe', 'rt'}:
        raise ValueError('datum must be kb, dfe or rt')
    
    if units not in {'m', 'ft'}:
        raise ValueError('units must be m or ft')
 
    try:
        elevation = float(elevation + 0)
    except TypeError:
        raise TypeError('elevation must be float')
        
    return {'datum': datum, 'units': units, 'elevation': elevation}


In [ ]:
get_header(datum='kb', units='m', elevation=12)

In [ ]:
def mdtotvd(deviation, method='mincurv'):
    """
    Calculate TVD from given method.
    
    Parameters
    ----------
    deviation: DataFrame in MD, incl, azi
    method: str, default 'tangential', {'tan', 'avtan', 'baltan', 'merc', 'radcurv', 'mincurv'}
            'tan' (tangential method),  'avtan' (average tangential method),
            'baltan' (balanced tangential method), 'merc' (mercury method),
            'radcurv' (radius of curvature method), 'mincurv' (minimum curvature method)
            method definitions: [Crain's Petrophysical Handbook](https://www.spec2000.net/19-dip13.htm)
            
    Returns
    -------
    DataFrame
        deviation survey converted to TVD, easting, northing
        
    To Do
    -----
    implement methods
            
    """

    return deviation

Source:  [Crain's Petrophysical Handbook](https://www.spec2000.net/19-dip13.htm)

Definitions:
  - East = easterly displacement (feet or meters) - negative = West
  - HAZ1 = hole azimuth at top of course (degrees)
  - HAZ2 = hole azimuth at bottom of course (degrees)
  - MD1 = measured depth at top of course (feet or meters)
  - MD2 = measured depth at bottom of course (feet or meters)
  - North = northerly displacement (feet or meters) - negative = South
  - TVD = true vertical depth (feet or meters)
  - WD1 = well deviation at top of course (degrees)
  - WD2 = well deviation at bottom of course (degrees)

The minimum curvature method, like the radius of curvature method, takes the space vectors defined by inclination and direction measurements and smoothes these onto the wellbore curve by the use of a ratio factor which is defined by the curvature (dog-leg) of the wellbore section. The method produces a circular arc as does the radius of the curvature. This is not, however, an assumption of the method, but a result of minimizing the total curvature within the physical constraints on a section of wellbore.

      1: DL = Arccos (Cos (WD2 - WD1) - Sin WD1 * Sin WD2 * (1 - Cos (HAZ2 - HAZ1)))
      2: CF = 2 / DL * (Tan (DL / 2)) * 0.017 453
      3: North = SUM ((MD2 - MD1)*((Sin WD1 * Cos HAZ1 + Sin WD2 * Cos HAZ2) / 2) * CF)
      4: East = SUM ((MD2 - MD1) * ((Sin WD1 * Sin HAZ1 + Sin WD2 * Sin HAZ2) / 2) * CF)
      5: TVD = SUM (((MD2 - MD1) * (Cos WD2 + Cos WD1) / 2) * CF)

Where:

  DL = dog leg severity (degrees)
  
  CF = curvature factor (radians)
  
  The term  0.017 453 converts degrees to radians.

In [ ]:
dev = read_deviation('../data/deviation.csv')
dev.head()

In [ ]:
dev['Inc[rad]'] = np.radians(dev['Inc[deg]'].values)
dev['Azi[rad]'] = np.radians(dev['Azi[deg]'].values)
dev.head()

In [ ]:
dev_diff = dev.diff()

In [ ]:
dev_diff.head()

In [ ]:
dev_diff.rename(index=str, columns={'MD[m]': 'delta_MD[m]',
                                    'Inc[deg]': 'delta_Inc[deg]',
                                    'Azi[deg]': 'delta_Azi[deg]',
                                    'Inc[rad]': 'delta_Inc[rad]',
                                    'Azi[rad]': 'delta_Azi[rad]',
                                    'Dogleg [deg/30m]': 'delta_Dogleg [deg/30m]',
                                    'TVD[m]':'delta_TVD[m]',
                                    'North[m]': 'delta_North[m]', 
                                    'East[m]': 'delta_East[m]',
                                   },
                inplace=True,
               )

In [ ]:
dev.reset_index(drop=True, inplace=True)
dev_diff.reset_index(drop=True, inplace=True)

In [ ]:
dev = pd.concat([dev, dev_diff], axis=1, join_axes=[dev_diff.index])
dev.head()

DL = Arccos (Cos (WD2 - WD1) - Sin WD1 * Sin WD2 * (1 - Cos (HAZ2 - HAZ1)))

In [ ]:
cos_incl = np.cos(dev['delta_Inc[rad]'].values[1:])

In [ ]:
incl_uppers = dev['Inc[rad]'].values[:-1]
incl_lowers = dev['Inc[rad]'].values[1:]

In [ ]:
sin_incl_product = np.sin(incl_uppers) * np.sin(incl_lowers)

In [ ]:
az_uppers = dev['Azi[rad]'].values[:-1]
az_lowers = dev['Azi[rad]'].values[1:]

In [ ]:
cos_azi_diff = 1 - np.cos(az_lowers - az_uppers)

In [ ]:
cos_incl.shape, sin_incl_product.shape, cos_azi_diff.shape

In [ ]:
dl_temp = np.degrees(np.arccos(cos_incl - sin_incl_product * (cos_azi_diff)))

In [ ]:
dev['DL'] = np.insert(dl_temp, 0, np.nan)
dev.head()

In [ ]:
fig, ax = plt.subplots(figsize=(6,8), nrows=1, ncols=1)

ax.plot(dev.DL, dev['MD[m]'])
ax.plot(dev['Dogleg [deg/30m]'], dev['MD[m]'])
ax.set_ylim(dev['MD[m]'].values.max() + 100, 0)
ax.set_title('Calculated vs imported DLS')
labels = ['Calculated DLS', 'Imported DLS']
plt.legend(labels)

plt.show()

CF = 2 / DL * (Tan (DL / 2)) * 0.017453

The term 0.017453 converts degrees to radians.

In [ ]:
dev.head()

In [ ]:
dev['CF'] = (2 / np.radians(dev['DL']) * np.tan(np.radians(dev['DL'])) / 2)
dev.head()

In [ ]:
fig, ax = plt.subplots(figsize=(6,8), nrows=1, ncols=1)

ax.plot(dev.CF, dev['MD[m]'])
ax.set_ylim(dev['MD[m]'].values.max() + 100, 0)
ax.set_title('CF')
labels = ['CF']

plt.show()

North = SUM ((MD2 - MD1)*((Sin WD1 * Cos HAZ1 + Sin WD2 * Cos HAZ2) / 2) * CF)

In [ ]:
dev.head()

In [ ]:
incl_uppers = dev['Inc[rad]'][:-1] 
incl_lowers = dev['Inc[rad]'][1:]
azi_uppers = dev['Azi[rad]'][:-1]
azi_lowers = dev['Azi[rad]'][1:]

In [ ]:
sin_cos_inc_azi = (np.sin(incl_uppers) * np.cos(azi_uppers) + np.sin(incl_lowers) * np.cos(azi_lowers)) / 2

In [ ]:
dev['northing[m]'] = dev['delta_MD[m]'].values * sin_cos_inc_azi * dev['CF']
dev['northing[m]'] = dev['northing[m]'].cumsum()
dev.head()

East = SUM ((MD2 - MD1) * ((Sin WD1 * Sin HAZ1 + Sin WD2 * Sin HAZ2) / 2) * CF)

In [ ]:
sin_sin_inc_azi = (np.sin(incl_uppers) * np.sin(azi_uppers) + np.sin(incl_lowers) * np.sin(azi_lowers)) / 2

In [ ]:
dev['easting[m]'] = dev['delta_MD[m]'] * sin_sin_inc_azi * dev['CF']
dev['easting[m]'] = dev['easting[m]'].cumsum()
dev.head()

In [ ]:
plt.plot(dev['easting[m]'], dev['northing[m]'])

TVD = SUM (((MD2 - MD1) * (Cos WD2 + Cos WD1) / 2) * CF)

In [ ]:
dev.head()

In [ ]:
incl_uppers = dev['Inc[rad]'].values[:-1]
incl_lowers = dev['Inc[rad]'].values[1:]

In [ ]:
tvd_temp = dev['delta_MD[m]'].values[1:] * (np.cos(incl_lowers) + np.cos(incl_uppers)) / 2 * dev['CF'].values[1:]
dev['TVD'] = np.insert(tvd_temp, 0, np.nan)
dev['TVD'] = dev['TVD'].cumsum()
dev.head()

In [ ]:
plt.plot(dev['TVD'])
plt.xlim(80, 0)
plt.ylim(2000, 0)
plt.title('TVD plot')

In [ ]:
dev.columns

In [ ]:
fig, ax = plt.subplots(figsize=(6,8), nrows=1, ncols=1)

ax.plot(dev.TVD, dev['MD[m]'])
ax.plot(dev['TVD[m]'], dev['MD[m]'])
ax.set_ylim(dev['MD[m]'].values.max() + 100, 0)
ax.set_title('TVD plot')
labels = ['Calculated TVD', 'Imported TVD']
plt.legend(labels)

plt.show()

    1: DL = Arccos (Cos (WD2 - WD1) - Sin WD1 * Sin WD2 * (1 - Cos (HAZ2 - HAZ1)))
    2: CF = 2 / DL * (Tan (DL / 2)) * 0.017 453
    3: North = SUM ((MD2 - MD1)*((Sin WD1 * Cos HAZ1 + Sin WD2 * Cos HAZ2) / 2) * CF)
    4: East = SUM ((MD2 - MD1) * ((Sin WD1 * Sin HAZ1 + Sin WD2 * Sin HAZ2) / 2) * CF)
    5: TVD = SUM (((MD2 - MD1) * (Cos WD2 + Cos WD1) / 2) * CF)

    DL = dog leg severity (degrees)
    CF = curvature factor (radians)
    East = easterly displacement (feet or meters) - negative = West
    HAZ1 = hole azimuth at top of course (degrees)
    HAZ2 = hole azimuth at bottom of course (degrees)
    MD1 = measured depth at top of course (feet or meters)
    MD2 = measured depth at bottom of course (feet or meters)
    North = northerly displacement (feet or meters) - negative = South
    TVD = true vertical depth (feet or meters)
    WD1 = well deviation at top of course (degrees)
    WD2 = well deviation at bottom of course (degrees)

    Depth = 3500 ft
    Inclination = 15 degree (I1)
    Azimuth = 20degree (Az1)

    Depth = 3600 ft
    Inclination = 25 degree (I2)
    Azimuth = 45 degree (Az2)

In [ ]:
md_test = np.array([3500, 3600])
incl_test = np.array([15, 25])
azi_test = np.array([20, 45])

In [ ]:
def min_curve_method(md, inc, azi):
    """
    Calculate TVD using minimum curvature method.

    Parameters
    ----------
    md: float, measured depth in m or ft
    inc: float, well deviation in degrees
    azi: float, well azimuth in degrees

    Returns
    -------
    Deviation converted to TVD, easting, northing
        TVD in m or feet,
        northing in m or feet,
        easting in m or feet
    Dogleg
        Dogleg angle in degrees

    ToDo
    ----
    Implement DLS
        Dogleg in degrees/100ft or degrees/30m
        Requires `.get_header()` ouput
    Implement surface location
        replace `np.insert([tvd, northing, easting], 0, 0)` with
        `np.insert([tvd, northing, easting], 0, <surface location>)`
    """
    # inputs are array-like
    md = np.asarray(md, dtype = np.float)
    inc = np.asarray(inc, dtype = np.float)
    azi = np.asarray(azi, dtype = np.float)

    # inputs are same shape
    if not (md.shape == inc.shape == azi.shape):
        raise ValueError('md, inc, and azi must be the same shape')

    # md array increases strictly at each step
    try:
        1 / bool(np.all(md[1:] > md[:-1]))
    except ZeroDivisionError:
        raise ZeroDivisionError('md must have strictly increasing values')

    # get units
    #norm = 100 if units == 'm' else 30

    # convert degrees to radians for numpy functions
    azi_r = np.deg2rad(azi)
    inc_r = np.deg2rad(inc)

    # extract upper and lower survey stations
    md_upper, md_lower = md[:-1], md[1:]
    incl_upper, incl_lower = inc_r[:-1], inc_r[1:]
    azi_upper, azi_lower = azi_r[:-1], azi_r[1:]

    # calculate dogleg
    dl = np.rad2deg(np.arccos(np.cos(incl_lower - incl_upper) -
                              (np.sin(incl_upper) * np.sin(incl_lower) *
                               (1 - np.cos(azi_lower - azi_upper)))))

    # calculate dls
    #dls = (dl * (norm / md_lower - md_upper))

    # ratio factor, correct for dl == 0 values
    rf = 2 / np.deg2rad(dl) * np.tan(np.deg2rad(dl)/2)
    rf = np.where(dl == 0., 1, rf)

    northing = np.cumsum((md_lower - md_upper) / 2 * (np.sin(incl_upper) * np.cos(azi_upper)
                                            + np.sin(incl_lower) * np.cos(azi_lower)) * rf)
    northing = np.insert(northing, 0, 0)

    easting = np.cumsum((md_lower - md_upper) / 2 * (np.sin(incl_upper) * np.sin(azi_upper)
                                            + np.sin(incl_lower) * np.sin(azi_lower)) * rf)
    easting = np.insert(easting, 0, 0)

    tvd = np.cumsum((md_lower - md_upper) / 2 * (np.cos(incl_upper) + np.cos(incl_lower)) * rf)
    tvd = np.insert(tvd, 0, 0)

    return tvd, northing, easting

In [ ]:
min_curve_method(md_test, incl_test, azi_test)

In [ ]:
tvd, northing, easting = min_curve_method(dev['MD[m]'].values, dev['Inc[deg]'].values, dev['Azi[deg]'].values)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.gca(projection='3d')

ax.plot(easting, northing, tvd, label='test well')
ax.invert_zaxis()
ax.legend()

In [ ]:
fig = plt.figure(figsize=(8,8))
plt.plot(easting, northing)

plt.gca().invert_yaxis()

plt.xlabel('MD [m]')
plt.ylabel('TVD [m]')
plt.show()

In [ ]:
def high_tan_method(md, inc, azi):
    """
    Calculate TVD using minimum curvature method.

    Parameters
    ----------
    md: float, measured depth in m or ft
    inc: float, well deviation in degrees
    azi: float, well azimuth in degrees

    Returns
    -------
    Deviation converted to TVD, easting, northing
        TVD in m or feet,
        northing in m or feet,
        easting in m or feet

    ToDo
    ----
    Implement surface location
        replace `np.insert([tvd, northing, easting], 0, 0)` with
        `np.insert([tvd, northing, easting], 0, <surface location>)`
    """
    # inputs are array-like
    md = np.asarray(md, dtype = np.float)
    inc = np.asarray(inc, dtype = np.float)
    azi = np.asarray(azi, dtype = np.float)

    # inputs are same shape
    if not (md.shape == inc.shape == azi.shape):
        raise ValueError('md, inc, and azi must be the same shape')

    # md array increases strictly at each step
    try:
        1 / bool(np.all(md[1:] > md[:-1]))
    except ZeroDivisionError:
        raise ZeroDivisionError('md must have strictly increasing values')

    # get units
    #norm = 100 if units == 'm' else 30

    # convert degrees to radians for numpy functions
    azi_r = np.deg2rad(azi)
    inc_r = np.deg2rad(inc)

    # extract upper and lower survey stations
    md_upper, md_lower = md[:-1], md[1:]
    incl_lower = inc_r[1:]
    azi_lower = azi_r[1:]

    northing = np.cumsum((md_lower - md_upper) * np.sin(incl_lower) * np.cos(azi_lower))
    northing = np.insert(northing, 0, 0)

    easting = np.cumsum((md_lower - md_upper) * np.sin(incl_lower) * np.sin(azi_lower))
    easting = np.insert(easting, 0, 0)

    tvd = np.cumsum((md_lower - md_upper) * np.cos(incl_lower))
    tvd = np.insert(tvd, 0, 0)

    return tvd, northing, easting

In [ ]:
tvd_ht, mN_ht, mE_ht = high_tan_method(dev['MD[m]'].values, dev['Inc[deg]'].values, dev['Azi[deg]'].values)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.gca(projection='3d')

ax.plot(easting, northing, tvd, label='test well')
ax.plot(mE_ht, mN_ht, tvd_ht, label='test well')
ax.invert_zaxis()
ax.legend()

In [ ]:
def low_tan_method(md, inc, azi):
    """
    Calculate TVD using minimum curvature method.

    Parameters
    ----------
    md: float, measured depth in m or ft
    inc: float, well deviation in degrees
    azi: float, well azimuth in degrees

    Returns
    -------
    Deviation converted to TVD, easting, northing
        TVD in m or feet,
        northing in m or feet,
        easting in m or feet

    ToDo
    ----
    Implement surface location
        replace `np.insert([tvd, northing, easting], 0, 0)` with
        `np.insert([tvd, northing, easting], 0, <surface location>)`
    """
    # inputs are array-like
    md = np.asarray(md, dtype = np.float)
    inc = np.asarray(inc, dtype = np.float)
    azi = np.asarray(azi, dtype = np.float)

    # inputs are same shape
    if not (md.shape == inc.shape == azi.shape):
        raise ValueError('md, inc, and azi must be the same shape')

    # md array increases strictly at each step
    try:
        1 / bool(np.all(md[1:] > md[:-1]))
    except ZeroDivisionError:
        raise ZeroDivisionError('md must have strictly increasing values')

    # get units
    #norm = 100 if units == 'm' else 30

    # convert degrees to radians for numpy functions
    azi_r = np.deg2rad(azi)
    inc_r = np.deg2rad(inc)

    # extract upper and lower survey stations
    md_upper, md_lower = md[:-1], md[1:]
    incl_upper = inc_r[:-1]
    azi_upper = azi_r[:-1]

    northing = np.cumsum((md_lower - md_upper) * np.sin(incl_upper) * np.cos(azi_upper))
    northing = np.insert(northing, 0, 0)

    easting = np.cumsum((md_lower - md_upper) * np.sin(incl_upper) * np.sin(azi_upper))
    easting = np.insert(easting, 0, 0)

    tvd = np.cumsum((md_lower - md_upper) * np.cos(incl_upper))
    tvd = np.insert(tvd, 0, 0)

    return tvd, northing, easting

In [ ]:
tvd_lt, mN_lt, mE_lt = low_tan_method(dev['MD[m]'].values, dev['Inc[deg]'].values, dev['Azi[deg]'].values)

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure()
ax = fig.gca(projection='3d')

ax.plot(easting, northing, tvd, label='min curve')
ax.plot(mE_ht, mN_ht, tvd_ht, label='high tan')
ax.plot(mE_lt, mN_lt, tvd_lt, label='low tan')
ax.invert_zaxis()
ax.legend()

-------
-------
Average tan method

      1: North = SUM ((MD2 - MD1) * Sin ((WD2 + WD1) / 2) * Cos ((HAZ2 + HAZ1) / 2))
      2: East = SUM ((MD2 - MD1) * Sin ((WD2 + WD1) / 2) * Sin ((HAZ2 + HAZ1) / 2))
      3: TVD = SUM ((MD2 - MD1) * Cos ((WD2 + WD1) / 2))